<a href="https://colab.research.google.com/github/fecsaba/automatic_photo/blob/main/auto_get_meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive**

In [1]:
import os

# Csatoljuk a Google Drive-ot
from google.colab import drive
drive.mount('/content/drive')
!pip install pypng
!pip install Pillow

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.5 MB/s eta 0:00:00


# ***Action***

```
'/content/drive/MyDrive/Images'
```



## Meta kiszedése

In [35]:
from PIL import PngImagePlugin, Image
import os
import json

class PngValidator:
    """PNG fájlok érvényességének ellenőrzésére szolgáló osztály.

    Ez az osztály felelős a PNG fájlok formátumának validálásáért. Ellenőrzi, hogy egy fájl
    valóban érvényes PNG formátumú-e, függetlenül a fájl kiterjesztésétől.

    Methods:
        is_valid_png(file_path): Ellenőrzi egy fájl PNG formátumát.
    """

    @staticmethod
    def is_valid_png(file_path):
        """Ellenőrzi, hogy a fájl valóban PNG formátumú-e.

        A metódus megnyitja a fájlt és ellenőrzi annak belső formátumát,
        nem csak a kiterjesztést vizsgálja.

        Args:
            file_path (str): A vizsgálandó fájl teljes elérési útja.

        Returns:
            bool: True ha a fájl valódi PNG formátumú, False egyébként.

        Example:
            >>> validator = PngValidator()
            >>> is_valid = validator.is_valid_png("kep.png")
            >>> print(is_valid)
            True
        """
        try:
            with Image.open(file_path) as img:
                return isinstance(img, PngImagePlugin.PngImageFile) and img.format == 'PNG'
        except Exception:
            return False

class PngFileHandler:
    """PNG fájlok és mappák kezelésére szolgáló osztály.

    Ez az osztály felelős a fájlrendszer műveletek végrehajtásáért,
    beleértve a mappák beolvasását és a PNG fájlok azonosítását.

    Attributes:
        directory_path (str): A feldolgozandó mappa elérési útja.

    Methods:
        set_directory_path(directory_path): Beállítja a feldolgozandó mappa útvonalát.
        get_png_files(): Összegyűjti az érvényes PNG fájlokat a megadott mappából.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngFileHandler osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.directory_path = directory_path

    def set_directory_path(self, directory_path):
        """Beállítja a feldolgozandó mappa útvonalát.

        Args:
            directory_path (str): A feldolgozandó mappa elérési útja.
        """
        self.directory_path = directory_path

    def get_png_files(self):
        """Összegyűjti az érvényes PNG fájlokat a megadott mappából.

        A metódus ellenőrzi a megadott mappában található összes fájlt,
        és csak azokat a fájlokat adja vissza, amelyek valóban PNG formátumúak.

        Returns:
            list: Érvényes PNG fájlok teljes elérési útjainak listája.

        Note:
            - Ha nincs megadva mappa útvonal, üres listával tér vissza
            - Ha a mappa nem létezik, üres listával tér vissza
            - Figyelmeztetést ad ki, ha talál olyan fájlt, ami .png kiterjesztésű,
              de nem valódi PNG formátumú
        """
        if not self.directory_path:
            print("Nincs megadva mappa útvonal!")
            return []

        if not os.path.exists(self.directory_path):
            print(f"A megadott mappa nem létezik: {self.directory_path}")
            return []

        png_files = []
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if PngValidator.is_valid_png(file_path):
                png_files.append(file_path)
            elif filename.lower().endswith('.png'):
                print(f"\nFigyelmeztetés: A fájl '.png' kiterjesztésű, de nem valódi PNG: {filename}")

        return png_files

class MetadataReader:
    """PNG fájlok metaadatainak olvasására szolgáló osztály.

    Ez az osztály felelős a PNG fájlok metaadatainak kinyeréséért és
    megjelenítéséért. A metaadatok között szerepelhetnek például a készítő,
    a létrehozás dátuma, és egyéb egyedi információk.

    Methods:
        read_metadata(file_path): Beolvassa és megjeleníti egy PNG fájl metaadatait.
    """

    @staticmethod
    def read_metadata(file_path):
        """Beolvassa egy PNG fájl metaadatait.

        Args:
            file_path (str): A PNG fájl teljes elérési útja.

        Returns:
            dict: A metaadatok szótár formátumban, ahol a kulcsok a metaadat mezők nevei,
                 az értékek pedig a hozzájuk tartozó értékek.
                 Hiba esetén None-nal tér vissza.
                 Ha nincsenek metaadatok, üres szótárral tér vissza.

        Note:
            A metódus a következő eseteket kezeli:
            - Sikeres beolvasás esetén kiírja a metaadatokat
            - Ha nincsenek metaadatok, ezt jelzi
            - Ha a fájl nem található, hibaüzenetet ad
            - Egyéb hibák esetén hibaüzenetet ad
        """
        try:
            with Image.open(file_path) as img:
                metadata = img.text
                if metadata:
                    print(f"\nMetaadatok a következő fájlhoz: {os.path.basename(file_path)}")
                    print("-" * 50)
                    for k, v in metadata.items():
                        print(f"**{k}**:\n{v}\n")
                    return metadata
                else:
                    print(f"\nNem találtunk metaadatokat a következő fájlban: {os.path.basename(file_path)}")
                    return {}

        except FileNotFoundError:
            print(f"A fájl nem található: {file_path}")
            return None
        except Exception as e:
            print(f"Hiba történt a következő fájlnál {os.path.basename(file_path)}: {str(e)}")
            return None

class PngMetadataProcessor:
    """PNG fájlok metaadatainak feldolgozására szolgáló fő osztály.

    Ez az osztály koordinálja a teljes feldolgozási folyamatot, összekapcsolva
    a fájlkezelést és a metaadat-olvasást. Egy megadott mappában található összes
    PNG fájlt feldolgozza és összegyűjti azok metaadatait.

    Attributes:
        file_handler (PngFileHandler): A fájlkezelésért felelős objektum.

    Methods:
        process_directory(): Feldolgozza a megadott mappában található összes PNG fájlt.
        save_results_to_json(results): Elmenti az eredményeket JSON fájlba.
    """

    def __init__(self, directory_path=None):
        """Inicializálja a PngMetadataProcessor osztályt.

        Args:
            directory_path (str, optional): A feldolgozandó mappa elérési útja.
                                          Alapértelmezett érték: None.
        """
        self.file_handler = PngFileHandler(directory_path)

    def process_directory(self):
        """Feldolgozza a megadott mappában található összes PNG fájlt.

        A metódus végigmegy a megadott mappában található összes PNG fájlon,
        és összegyűjti azok metaadatait. A folyamat során:
        1. Azonosítja az érvényes PNG fájlokat
        2. Beolvassa minden fájl metaadatait
        3. Összesíti az eredményeket

        Returns:
            dict: Egy szótár, ahol a kulcsok a fájlnevek, az értékek pedig
                 a hozzájuk tartozó metaadatok szótár formátumban.
        """
        print(f"\nPNG fájlok keresése a következő mappában: {self.file_handler.directory_path}")
        print("=" * 70)

        png_files = self.file_handler.get_png_files()
        results = {}

        for file_path in png_files:
            metadata = MetadataReader.read_metadata(file_path)
            if metadata:
                results[os.path.basename(file_path)] = metadata

        if not results:
            print("\nNem találtunk érvényes PNG fájlokat a megadott mappában.")

        return results

    def save_results_to_json(self, results):
        """Elmenti az eredményeket JSON fájlba.

        Args:
            results (dict): A mentendő metaadatok szótár formátumban.

        Note:
            A fájl neve 'png_metadata_results.json' lesz, és az Image mappába kerül mentésre.
            Ha a fájl már létezik, felülírja azt.
        """
        if not results:
            print("\nNincs menthető adat.")
            return

        output_path = os.path.join(self.file_handler.directory_path, "png_metadata_results.json")
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            print(f"\nAz eredmények sikeresen mentve: {output_path}")
        except Exception as e:
            print(f"\nHiba történt a fájl mentése közben: {str(e)}")

# Példa használat és tesztelés
if __name__ == "__main__":
    # Példa a program használatára
    processor = PngMetadataProcessor("/content/drive/MyDrive/Image/")
    all_metadata = processor.process_directory()

    # Eredmények mentése JSON fájlba
    processor.save_results_to_json(all_metadata)

    # Összesítés
    if all_metadata:
        print("\nÖsszesítés:")
        print("=" * 70)
        print(f"Összesen {len(all_metadata)} érvényes PNG fájl metaadatait olvastuk be.")


PNG fájlok keresése a következő mappában: /content/drive/MyDrive/Image/

Metaadatok a következő fájlhoz: ComfyUI_00001_ (2).png
--------------------------------------------------
**parameters**:
A beautiful blonde woman walking in an urban city setting, her long hair flowing elegantly in the wind, soft natural lighting, cinematic composition, highly detailed facial features, photorealistic style with a touch of hyperrealism, dynamic motion capture, autumn leaves gently swirling around her, crisp and vibrant colors, 8k resolution, art by Greg Rutkowski and Loish. High detail on textures, sharp focus on the woman's face and hair, depth of field effect to emphasize the subject, warm golden hour tones, clean and polished finish, inspired by high-fashion photography and film stills. Ultra-detailed skin texture and lifelike eyes, Soft bokeh effects for lights in the background,perfection style, Cinematic Photography style,
Negative prompt: 
Steps: 20, Sampler: euler_simple, Seed: 6273623346

## Pozitív prompt kiszedése

### Leírás kiválasztása

In [44]:
import json
import re
import sys

def extract_pos_from_parameters(input_json_path, output_json_path):
    """
    Extracts the 'Pos' value from the 'parameters' string in the input JSON and saves it to a new JSON file.
    """
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_json_path}")
        return False
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in input file {input_json_path}")
        return False
    except Exception as e:
        print(f"Error: An unexpected error occurred: {type(e).__name__} - {e}")
        return False

    pos_data = {}
    pos_pattern = re.compile(r'Pos:\s*([\s\S]*?)(?:,\n|$)')  # Regex pattern to extract the Pos value

    for image_filename, image_data in data.items():
        if 'parameters' in image_data and isinstance(image_data['parameters'], str):
            match = pos_pattern.search(image_data['parameters'])
            if match:
                pos_value = match.group(1).strip()  # Get the matched Pos value and strip whitespace
                pos_data[image_filename] = pos_value
            else:
                print(f"Warning: No 'Pos' key found in parameters for {image_filename}")
                pos_data[image_filename] = ""  # Üres string, ha nincs 'Pos'
        else:
            print(f"Warning: No 'parameters' string found for {image_filename}")
            pos_data[image_filename] = ""  # Üres string, ha nincs 'parameters'

    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(pos_data, f, indent=2)  # Szebb JSON formázás
    except Exception as e:
        print(f"Error: Could not write to output file {output_json_path}: {e}")
        return False

    return True

if __name__ == '__main__':
    if len(sys.argv) != 3:
        print("Usage: python3 extract_pos.py <input_json_file> <output_json_file>")
        sys.exit(1)

    input_file = "/content/drive/MyDrive/Image/png_metadata_results.json"
    output_file = "/content/drive/MyDrive/Image/desc.json"

    if extract_pos_from_parameters(input_file, output_file):
        print(f"Successfully extracted 'Pos' values to {output_file}")
    else:
        print("Extraction failed.")

Successfully extracted 'Pos' values to /content/drive/MyDrive/Image/desc.json


### Szöveg generálás

### Tisztítás

In [45]:
# Szükséges könyvtárak importálása
import json
import os
from google.colab import drive, files

# 1. Google Drive csatolása
drive.mount('/content/drive')

# 2. Projekt könyvtár és fájlok beállítása
project_dir = '/content/drive/MyDrive/Image'
input_filename = os.path.join(project_dir, 'desc.json')
output_filename = os.path.join(project_dir, 'cleaned_output.json')

# 3. Ellenőrzés, hogy a desc.json létezik-e
if not os.path.exists(input_filename):
    print("Hiba: A 'desc.json' nem található a projektkönyvtárban!")
    print("Kérlek, töltsd fel a fájlt, ha szükséges!")
    uploaded = files.upload()
    for fname, content in uploaded.items():
        with open(input_filename, 'wb') as f:
            f.write(content)
    print("Fájl feltöltve a projektkönyvtárba!")
else:
    print("A 'desc.json' megtalálva a projektkönyvtárban, folytatom a feldolgozást...")

# 4. JSON fájl betöltése
with open(input_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 5. Promptok tisztítása függvény
def clean_prompt(prompt):
    # Perjelek cseréje szóközre
    cleaned = prompt.replace('/', ' ')
    # Zárójelek eltávolítása
    cleaned = cleaned.replace('(', '').replace(')', '')
    # Többszörös szóközök eltávolítása
    cleaned = ' '.join(cleaned.split())
    return cleaned.strip()

# 6. Minden prompt megtisztítása
cleaned_data = {}
for filename, prompt in data.items():
    cleaned_prompt = clean_prompt(prompt)
    cleaned_data[filename] = cleaned_prompt

# 7. Megtisztított JSON mentése a projektkönyvtárba
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

# 8. Az új fájl letöltése
print("A megtisztított JSON fájl elmentve a projektkönyvtárba ('cleaned_output.json'), és letöltheted!")
files.download(output_filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
A 'desc.json' megtalálva a projektkönyvtárban, folytatom a feldolgozást...
A megtisztított JSON fájl elmentve a projektkönyvtárba ('cleaned_output.json'), és letöltheted!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **SEO generation**

In [46]:
# Szükséges könyvtárak importálása
from google.colab import drive, files
import json
import os
from transformers import pipeline

class FileManager:
    """Osztály a Google Drive fájlkezeléséhez és JSON műveletekhez."""

    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        """Inicializálja a fájlkezelőt a megadott könyvtárral és fájlnévvel."""
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = None

    def mount_drive(self):
        """Csatolja a Google Drive-ot a Colab környezethez."""
        print("Csatolás a Google Drive-hoz...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        """Betölti a JSON fájlt, ha létezik, különben hibát jelez."""
        if not os.path.exists(self.filename):
            print(f"Hiba: A '{self.filename}' nem található a projektkönyvtárban!")
            raise FileNotFoundError("Kérlek, futtasd az előző szkriptet először!")
        print(f"A '{self.filename}' megtalálva, betöltés...")
        with open(self.filename, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        return self.data

    def save_json(self):
        """Visszaírja a frissített adatokat a JSON fájlba."""
        print("Frissített fájl mentése...")
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

    def download_file(self):
        """Letölti a frissített JSON fájlt a böngészőbe."""
        print(f"A '{self.filename}' frissítve, letöltheted!")
        files.download(self.filename)

class SEOGenerator:
    """Osztály SEO leírások generálására, stílus- és technikai részletek nélkül."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a generátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a leírásokhoz...")
        self.summarizer = pipeline("summarization", model=model_name)
        # Stílus- és technikai kulcsszavak kiszűrése
        self.technical_style_terms = {
            "8k", "resolution", "cinematic", "photorealistic", "hyperrealism", "dynamic", "motion", "capture",
            "crisp", "vibrant", "colors", "high", "detail", "textures", "sharp", "focus", "depth", "field",
            "effect", "emphasize", "subject", "warm", "golden", "hour", "tones", "clean", "polished", "finish",
            "inspired", "high-fashion", "photography", "film", "stills", "ultra-detailed", "texture", "lifelike",
            "eyes", "soft", "bokeh", "effects", "lights", "background", "perfection", "style", "DSLR", "photo",
            "Fujifilm", "X-T4", "f", "2.8", "natural", "lighting", "composition", "variety", "conditions",
            "Adobe", "After", "Effects", "software", "realistic", "striking", "visible", "enhance"
        }

    def filter_technical_style_terms(self, prompt):
        """Eltávolítja a stílus- és technikai kulcsszavakat a promptból."""
        words = prompt.lower().split()
        filtered_words = [word for word in words if word not in self.technical_style_terms]
        return " ".join(filtered_words)

    def generate_seo_description(self, prompt):
        """Generál egy maximum 500 karakteres SEO leírást a promptból, stílus- és technikai részletek nélkül."""
        print(f"SEO leírás generálása a promptból (hossz: {len(prompt)} karakter)...")
        # Technikai és stílus részletek kiszűrése
        filtered_prompt = self.filter_technical_style_terms(prompt)
        print(f"Szűrt prompt: '{filtered_prompt}' ({len(filtered_prompt)} karakter)")
        # Rövid összefoglaló generálása, a fő témára fókuszálva
        summary = self.summarizer(filtered_prompt, max_length=80, min_length=20, do_sample=False)[0]['summary_text']
        seo_desc = summary.strip()[:500]
        # Redundancia eltávolítása
        words = seo_desc.split()
        seo_desc = " ".join(sorted(set(words), key=words.index))[:500]
        print(f"Generált leírás: '{seo_desc}' ({len(seo_desc)} karakter)")
        return seo_desc

class SEOTitleGenerator:
    """Osztály SEO címek generálására a promptból, a fő témára fókuszálva."""

    def __init__(self, model_name='facebook/bart-large-cnn'):
        """Inicializálja a címgenerátort a megadott modellel."""
        print(f"Betöltés a '{model_name}' modellhez a címekhez...")
        self.summarizer = pipeline("summarization", model=model_name)

    def generate_seo_title(self, prompt):
        """Generál egy maximum 100 karakteres SEO címet a promptból, a fő témára koncentrálva."""
        print(f"SEO cím generálása a promptból (hossz: {len(prompt)} karakter)...")
        short_prompt = " ".join(prompt.split()[:50])
        input_text = f"Summarize the main subject of: {short_prompt}. Focus on the key subject, keep it short, max 100 characters."
        summary = self.summarizer(input_text, max_length=15, min_length=5, do_sample=False)[0]['summary_text']
        seo_title = summary.strip()[:100]
        if len(seo_title) == 100 and not seo_title.endswith('.'):
            seo_title = seo_title[:-1] + '.'
        print(f"Generált cím: '{seo_title}' ({len(seo_title)} karakter)")
        return seo_title

def main():
    """Fő függvény az osztályok használatához és a folyamat vezérléséhez."""
    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()

    seo_generator = SEOGenerator()
    print("SEO leírások ellenőrzése és generálása...")
    for filename, entry in data.items():
        if isinstance(entry, dict):
            if "seo_description" not in entry or len(entry["seo_description"]) > 500:
                print(f"Frissítés szükséges: {filename}...")
                entry["seo_description"] = seo_generator.generate_seo_description(entry["prompt"])
        else:
            print(f"Átalakítás szótárrá és leírás generálása: {filename}...")
            prompt = entry
            seo_desc = seo_generator.generate_seo_description(prompt)
            data[filename] = {"prompt": prompt, "seo_description": seo_desc}

    seo_title_generator = SEOTitleGenerator()
    print("SEO címek ellenőrzése és generálása...")
    for filename, entry in data.items():
        print(f"Feldolgozás: {filename}...")
        seo_title = seo_title_generator.generate_seo_title(entry["prompt"])
        entry["seo_title"] = seo_title

    file_manager.data = data
    file_manager.save_json()
    file_manager.download_file()

if __name__ == "__main__":
    main()

Csatolás a Google Drive-hoz...
Mounted at /content/drive
A '/content/drive/MyDrive/Image/cleaned_output.json' megtalálva, betöltés...
Betöltés a 'facebook/bart-large-cnn' modellhez a leírásokhoz...


Device set to use cpu
Your max_length is set to 80, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


SEO leírások ellenőrzése és generálása...
Átalakítás szótárrá és leírás generálása: ComfyUI_00001_ (2).png...
SEO leírás generálása a promptból (hossz: 717 karakter)...
Szűrt prompt: 'a beautiful blonde woman walking in an urban city setting her long hair flowing elegantly in the wind highly detailed facial features with a touch of autumn leaves gently swirling around her and art by greg rutkowski and loish. on on the woman's face and hair of to the and by and stills. skin and for in the' (308 karakter)


Your max_length is set to 80, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


Generált leírás: 'A beautiful blonde woman walking in an urban city setting her long hair flowing elegantly the wind. highly detailed facial features with a touch of autumn leaves gently swirling around her.' (189 karakter)
Átalakítás szótárrá és leírás generálása: ComfyUI_00005_.png...
SEO leírás generálása a promptból (hossz: 457 karakter)...
Szűrt prompt: 'a white cat with bright green peeking out from behind a torn piece of paper highly detailed fur playful and curious expression paws in a playful pose and set against a plain white whiskers and claws art by albert hirai and david revoy. around the edges to depth.' (262 karakter)


...

Generált leírás: 'A white cat with bright green peeking out from behind a torn piece of black paper. highly detailed fur playful and curious expression paws in pose.' (147 karakter)
Átalakítás szótárrá és leírás generálása: ComfyUI_00006_ (1).png...
SEO leírás generálása a promptból (hossz: 422 karakter)...
Szűrt prompt: 'dslr photo:1.3 professional pet grooming salon interior retriever with wet fluffy fur water droplets fur calico cat on marble table sunlight stainless steel grooming tools ceramic shampoo bottles microfiber towels pastel mint green walls blush pink accents of fujifilm x-t4' (273 karakter)
Generált leírás: '3 professional pet grooming salon interior. Pastel mint green walls blush pink accents of fujifilm x-t4.' (104 karakter)
Betöltés a 'facebook/bart-large-cnn' modellhez a címekhez...


Device set to use cpu


SEO címek ellenőrzése és generálása...
Feldolgozás: ComfyUI_00001_ (2).png...
SEO cím generálása a promptból (hossz: 717 karakter)...
Generált cím: 'A beautiful blonde woman walking in an urban city setting her long' (66 karakter)
Feldolgozás: ComfyUI_00005_.png...
SEO cím generálása a promptból (hossz: 457 karakter)...
Generált cím: 'White cat with striking bright green eyes peeking out from behind' (65 karakter)
Feldolgozás: ComfyUI_00013_.png...
SEO cím generálása a promptból (hossz: 463 karakter)...
Generált cím: 'White cat with striking bright green eyes peeking out from behind' (65 karakter)
Feldolgozás: ComfyUI_00006_ (1).png...
SEO cím generálása a promptból (hossz: 422 karakter)...
Generált cím: 'Fujifilm X-T4 is an ultra' (25 karakter)
Frissített fájl mentése...
A '/content/drive/MyDrive/Image/cleaned_output.json' frissítve, letöltheted!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Képfelismerés

In [ ]:
from PIL import Image
import json
import os
from google.colab import drive
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from datetime import datetime

class FileManager:
    def __init__(self, project_dir='/content/drive/MyDrive/Image', filename='cleaned_output.json'):
        self.project_dir = project_dir
        self.filename = os.path.join(project_dir, filename)
        self.data = {}

    def mount_drive(self):
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=True)

    def load_json(self):
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r', encoding='utf-8') as f:
                    self.data = json.load(f)
            except json.JSONDecodeError:
                print(f"Figyelmeztetés: '{self.filename}' érvénytelen JSON fájl.")
                self.data = {}
        else:
            print(f"'{self.filename}' nem található. Új fájl lesz létrehozva.")
        return self.data

    def save_json(self):
        try:
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump(self.data, f, ensure_ascii=False, indent=2)
            print(f"'{self.filename}' mentve!")
        except Exception as e:
            print(f"Hiba a fájl mentése közben: {e}")

    def list_images(self):
        if not os.path.exists(self.project_dir):
            print(f"Hiba: '{self.project_dir}' könyvtár nem található.")
            return []
        return [f for f in os.listdir(self.project_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

class ImageDescriber:
    def __init__(self, processor, model):
        self.processor = processor
        self.model = model

    def load_image(self, image_path):
        try:
            return Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"Hiba a(z) '{image_path}' betöltése közben: {e}")
            return None

    def generate_description(self, image_path):
        image = self.load_image(image_path)
        if image is None:
            return "Nem sikerült betölteni a képet."
        inputs = self.processor(image, return_tensors="pt").to('cuda')
        out = self.model.generate(
            **inputs,
            max_new_tokens=500,  # Célzottan 450-500 karakter körüli leírás
            num_beams=3,         # Több variáció a részletesebb leírásért
            length_penalty=2.8,  # Erősebb ösztönzés hosszabb szövegre
            no_repeat_ngram_size=2,
            do_sample=False,     # Következetes generálás
            early_stopping=False # Ne álljon meg korán
        )
        desc = self.processor.decode(out[0], skip_special_tokens=True)
        return desc

def main():
    if not torch.cuda.is_available():
        print("Figyelmeztetés: GPU nem elérhető. A feldolgozás lassabb lehet.")
    else:
        print("GPU észlelve. GPU-t használjuk a feldolgozáshoz.")

    file_manager = FileManager()
    file_manager.mount_drive()
    data = file_manager.load_json()
    image_files = file_manager.list_images()

    if not image_files:
        print("Nincsenek képek a megadott könyvtárban.")
        return

    print(f"Talált képek: {image_files}")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to('cuda')
    describer = ImageDescriber(processor, model)

    for filename in image_files:
        image_path = os.path.join(file_manager.project_dir, filename)
        print(f"Feldolgozás: {filename}...")

        try:
            prompt = data.get(filename, {}).get("prompt", None)  # Csak tárolásra, nem használatra
            image_desc = describer.generate_description(image_path)
            print(f"{filename} leírása: {image_desc} (Hossz: {len(image_desc)})")

            processed_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if filename not in data:
                data[filename] = {}
            data[filename].update({
                "prompt": prompt or "Nincs megadva prompt",
                "image_description": image_desc,
                "link": "",
                "processed_at": processed_at,
                "uploaded_at": ""
            })
        except Exception as e:
            print(f"Hiba a(z) '{filename}' feldolgozása közben: {e}")

    file_manager.data = data
    file_manager.save_json()

if __name__ == "__main__":
    main()

Figyelmeztetés: GPU nem érhető el. A feldolgozás lassabb lehet.
Google Drive csatolása...
Mounted at /content/drive
Talált képek: ['ComfyUI_00001_ (2).png', 'ComfyUI_00005_.png', 'ComfyUI_00013_.png', 'ComfyUI_00006_ (1).png']


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Régi

In [38]:
import json
import re
import sys

def extract_pos_from_parameters(input_json_path, output_json_path):
    """
    Extracts the 'Pos' value from the 'parameters' string in the input JSON and saves it to a new JSON file.
    """
    try:
        with open(input_json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_json_path}")
        return False
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in input file {input_json_path}")
        return False
    except Exception as e:
        print(f"Error: An unexpected error occurred: {type(e).__name__} - {e}")
        return False

    pos_data = {}
    pos_pattern = re.compile(r'Pos:\s*([\s\S]*?)(?:,\n|$)')  # Regex pattern to extract the Pos value

    for image_filename, image_data in data.items():
        if 'parameters' in image_data and isinstance(image_data['parameters'], str):
            match = pos_pattern.search(image_data['parameters'])
            if match:
                pos_value = match.group(1).strip()  # Get the matched Pos value and strip whitespace
                pos_data[image_filename] = pos_value
            else:
                print(f"Warning: No 'Pos' key found in parameters for {image_filename}")
                pos_data[image_filename] = ""  # Üres string, ha nincs 'Pos'
        else:
            print(f"Warning: No 'parameters' string found for {image_filename}")
            pos_data[image_filename] = ""  # Üres string, ha nincs 'parameters'

    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(pos_data, f, indent=2)  # Szebb JSON formázás
    except Exception as e:
        print(f"Error: Could not write to output file {output_json_path}: {e}")
        return False

    return True

if __name__ == '__main__':
    if len(sys.argv) != 3:
        print("Usage: python3 extract_pos.py <input_json_file> <output_json_file>")
        sys.exit(1)

    input_file = "/content/drive/MyDrive/Image/png_metadata_results.json"
    output_file = "/content/drive/MyDrive/Image/desc.json"

    if extract_pos_from_parameters(input_file, output_file):
        print(f"Successfully extracted 'Pos' values to {output_file}")
    else:
        print("Extraction failed.")

Successfully extracted 'Pos' values to /content/drive/MyDrive/Image/desc.json


## Előkészítés feltöltéshez

In [21]:
import json
from datetime import datetime
from transformers import pipeline
from google.colab import drive  # Import for Google Drive integration
from dotenv import load_dotenv  # For handling environment variables
import os
import re

# Mount Google Drive
drive.mount('/content/drive')

# Define the input and output directories
INPUT_FILE_PATH = "/content/drive/MyDrive/Image/desc.json"
OUTPUT_DIRECTORY = "/content/drive/MyDrive/Image"

# Load environment variables from .env file
load_dotenv()

# Securely get the Hugging Face token from the environment variable
HF_TOKEN = os.getenv("HF_TOKEN") or None

# Load input data from desc.json
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as input_file:
            return json.load(input_file)
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return {}
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not valid JSON.")
        return {}

data = load_data(INPUT_FILE_PATH)

if not data:
    print("No data to process. Exiting.")
else:
    # Language model for rewriting descriptions
    rewriter = pipeline("text2text-generation", model="t5-base")

    # Keyword extraction model (using bloomberg/KeyBART)
    keyword_extractor = pipeline("text2text-generation", model="bloomberg/KeyBART", tokenizer="bloomberg/KeyBART", token=HF_TOKEN)

    # Function to clean and rewrite the description
    def clean_and_rewrite_description(description):
        """
        Clean up the raw description and rewrite it into a coherent, SEO-friendly format.
        """
        # Remove unwanted characters and split into parts
        cleaned_parts = [part.strip().capitalize() for part in re.sub(r'[()\[\]{}<>/]', '', description).split(",") if part.strip()]

        # Use the language model to rewrite the description
        rewritten_description = rewriter(". ".join(cleaned_parts), max_length=500, truncation=True)[0]['generated_text']

        # Post-process the rewritten description: ensure proper capitalization and no repetitions
        final_description_parts = []
        seen_sentences = set()
        for sentence in rewritten_description.split("."):
            sentence = sentence.strip()
            if sentence and sentence not in seen_sentences and len(sentence) > 10:
                seen_sentences.add(sentence)
                final_description_parts.append(sentence.capitalize())

        # Combine rewritten description with character limit
        final_description = []
        current_length = 0
        for sentence in final_description_parts:
            if current_length + len(sentence) + 1 <= 500:
                final_description.append(sentence)
                current_length += len(sentence) + 1
            else:
                break

        return ". ".join(final_description).strip()

    # Function to check relevance of a tag to the description
    def is_relevant_tag(tag, description):
        """
        Check if the tag is relevant to the description.
        A tag is considered relevant if it appears in the cleaned description or closely related content.
        """
        normalized_description = re.sub(r'[^\w\s]', '', description.lower())  # Remove punctuation, make lowercase
        normalized_tag = re.sub(r'[^\w\s]', '', tag.lower())  # Remove punctuation, make lowercase

        # Check if the tag or its key words are present in the description
        return normalized_tag in normalized_description or any(word in normalized_description for word in normalized_tag.split())

    # Clean and format tags
    def clean_and_format_tags(raw_keywords, description):
        """
        Clean and format the raw keywords/tags based on relevance and formatting rules.
        """
        tags = []
        seen_tags = set()
        for tag in raw_keywords:
            # Normalize tag: lowercase, remove punctuation
            tag = tag.strip().lower().replace(";", "").replace(".", "").replace("(", "").replace(")", "")

            # Include only unique, relevant, and properly formatted tags
            if tag and tag not in seen_tags and is_relevant_tag(tag, description):
                seen_tags.add(tag)
                tags.append(tag.capitalize())  # Capitalize the first letter

        # Limit to 5 tags and join with commas
        return ", ".join(tags[:5])

    # Process data function
    def process_data(data):
        processed_data = {}
        for filename, description in data.items():
            # Clean up the description: remove / and styles, capitalize sentences
            cleaned_description = ". ".join([part.strip().capitalize() for part in description.split("/") if part.strip()])

            # Generate SEO-friendly title
            seo_title = "Photorealistic Woman in Urban Autumn Setting | High-Resolution Artwork" if "woman" in cleaned_description.lower() else (
                "Lifelike White Cat with Bright Green Eyes | Detailed Digital Illustration" if "cat" in cleaned_description.lower() else "High-Quality Pet Grooming Photography | Nature Illustration"
            )
            cta_text = "❤️ Discover this stunning artwork and bring elegance to your space today!" if "woman" in cleaned_description.lower() else (
                "💚 Admire the charm of this adorable feline masterpiece and make it yours now!" if "cat" in cleaned_description.lower() else "🐾 Explore the beauty of pet grooming through this incredible photography!"
            )

            # Rewrite description using language model
            rewritten_description = clean_and_rewrite_description(cleaned_description)

            # Extract keywords/tags using bloomberg/KeyBART and clean them
            try:
                raw_keywords = keyword_extractor(rewritten_description, max_length=50, truncation=True)[0]['generated_text'].split(", ")

                # Clean and filter tags based on relevance and formatting
                tags_str = clean_and_format_tags(raw_keywords, rewritten_description)
            except Exception as e:
                print(f"Error extracting keywords for {filename}: {e}")
                tags_str = ""  # Fallback to empty tags if extraction fails

            # Add an empty field for the posted date
            processed_data[filename] = {
                "seo_title": seo_title,
                "description": rewritten_description,
                "cta": cta_text,
                "tags": tags_str,  # Save tags as a comma-separated string
                "processing_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "posted_date": ""  # Empty field for when the image is posted
            }

        return processed_data

    # Save processed data to JSON file
    def save_to_json(data, file_path):
        try:
            with open(file_path, "w", encoding="utf-8") as json_file:
                json.dump(data, json_file, indent=4, ensure_ascii=False)
            print(f"JSON file saved successfully at: {file_path}")
        except Exception as e:
            print(f"Error saving JSON file: {e}")

    # Main processing logic
    processed_data = process_data(data)
    OUTPUT_FILE_PATH = f"{OUTPUT_DIRECTORY}/processed_art_data.json"
    save_to_json(processed_data, OUTPUT_FILE_PATH)

    print(f"Processing completed. Output saved to: {OUTPUT_FILE_PATH}")

KeyboardInterrupt: 